In [2]:
%cd ../src

c:\Users\LENOVO\Desktop\facebook_crawler\src


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import re
from time import sleep
import getpass


option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
# option.add_argument("--auto-open-devtools-for-tabs")
option.add_argument("--disable-extensions")
option.add_experimental_option(
    "prefs", {"profile.default_content_setting_values.notifications": 1}
)
browser = webdriver.Chrome(options=option)

from selenium.webdriver.common.action_chains import ActionChains
actions = ActionChains(browser)

base_url = "https://www.facebook.com/"
browser.get(base_url)

In [14]:
username = "yelan482@gmail.com"
password  = 'Linhcute542002'

from utils.login import login
login(browser, username, password)

Success Login


In [15]:
import group
import user_profile
from tqdm import tqdm
import time 

group_crawler = group.Group(browser=browser)
url_group = 'https://www.facebook.com/groups/932212904285349'
user_list = group_crawler.get_url_from_members(url_group, max_user=50)

user_inf_crawler =  user_profile.UserInfo(browser)
browser.quit()

Start Crawling User from Group: KHỐI 12 - TRƯỜNG THPT NGUYỄN HUỆ
===37/50
===77/50


100%|██████████| 50/50 [00:02<00:00, 19.76it/s]


# 1. MultiThread for Crawl User_info from list of UserURLs

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import threading
from utils.login import login
import user_profile


def initialize_driver():
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_experimental_option(
        "prefs", {"profile.default_content_setting_values.notifications": 1}
    )
    return webdriver.Chrome(options=option)

def crawl_user_info(driver, user_list, start_index, chunk_size, result):
    user_crawl =  user_profile.UserInfo(driver)
    for i in range(start_index, start_index+chunk_size):
        user_info = user_crawl.get_user_info(user_list[i])
        result[i] = user_info


username = "yelan482@gmail.com"
password = 'Linhcute542002'


base_url = 'https://www.facebook.com/'
drivers = [initialize_driver() for _ in range(5)]

for driver in drivers:
    driver.get(base_url)
    login(driver, username, password)

result = [None] * len(user_list)

chunk_size = len(user_list) // 5
start = 0

threads = []
for driver in drivers:
    thread = threading.Thread(target=crawl_user_info, args=(driver, user_list, start, chunk_size, result))
    threads.append(thread)
    start += chunk_size

start_time = time.perf_counter()

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

print(time.perf_counter() - start_time, 'seconds')

for driver in drivers:
    driver.quit()

Success Login
Success Login
Success Login
Success Login
Success Login
271.7883135000002 seconds


# 2. MultiThread for Crawl multiple group members from list of groups

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import threading
from utils.login import login
import user_profile
import group

def initialize_driver():
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_experimental_option(
        "prefs", {"profile.default_content_setting_values.notifications": 1}
    )
    return webdriver.Chrome(options=option)

def crawl_group_member(driver, group_list, start_index, chunk_size, result):
    group_crawler = group.Group(browser=driver)
    for i in range(start_index, start_index+chunk_size):
        try:
            user_list = group_crawler.get_url_from_members(group_list[i], max_user=100)
        except:
            user_list = None
        result[i] = user_list


username = "lth2112002@gmail.com"
password = 'Baoanh123'


group_links=[
    'https://www.facebook.com/groups/932212904285349',
    'https://www.facebook.com/groups/1986195691670003',
    'https://www.facebook.com/groups/263503162332380',
    'https://www.facebook.com/groups/flexdenhoithocuoicung',
    'https://www.facebook.com/groups/nthehatoansongsong',
]

base_url = 'https://www.facebook.com/'
drivers = [initialize_driver() for _ in range(5)]

for driver in drivers:
    driver.get(base_url)
    login(driver, username, password)

result = [None] * len(group_links)

chunk_size = len(group_links) // 5   # Max = 5
start = 0

threads = []
for driver in drivers:
    thread = threading.Thread(target=crawl_group_member, args=(driver, group_links, start, chunk_size, result))
    threads.append(thread)
    start += chunk_size

start_time = time.perf_counter()

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

print(time.perf_counter() - start_time, 'seconds')

for driver in drivers:
    driver.quit()

Success Login
Success Login
Success Login
Success Login
Success Login
Start Crawling User from Group: CLB Sáng tạo trẻ - Trường THPT A Lưới
===73/100
===93/100
===103/100


100%|██████████| 100/100 [00:01<00:00, 62.72it/s]


20.2755345999999 seconds


In [7]:
result

[None,
 ['https://www.facebook.com/profile.php?id=100071684331377',
  'https://www.facebook.com/profile.php?id=100015733534289',
  'https://www.facebook.com/profile.php?id=100013886446373',
  'https://www.facebook.com/profile.php?id=61555335785751',
  'https://www.facebook.com/profile.php?id=100092380986251',
  'https://www.facebook.com/profile.php?id=100082605951622',
  'https://www.facebook.com/profile.php?id=100003046968242',
  'https://www.facebook.com/profile.php?id=100003677062318',
  'https://www.facebook.com/profile.php?id=100008105701061',
  'https://www.facebook.com/profile.php?id=100066320130406',
  'https://www.facebook.com/profile.php?id=100077485805711',
  'https://www.facebook.com/profile.php?id=61551622838976',
  'https://www.facebook.com/profile.php?id=100076634932874',
  'https://www.facebook.com/profile.php?id=100084516473525',
  'https://www.facebook.com/profile.php?id=100084144864498',
  'https://www.facebook.com/profile.php?id=100031024537497',
  'https://www.face

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random
import threading
from utils.login import login
import user_profile
import group

username = "lth2112002@gmail.com"
password = 'Baoanh123'


group_links=[
'https://www.facebook.com/groups/932212904285349',
'https://www.facebook.com/groups/1986195691670003',
    'https://www.facebook.com/groups/263503162332380',
    'https://www.facebook.com/groups/flexdenhoithocuoicung',
    'https://www.facebook.com/groups/nthehatoansongsong',
]

base_url = 'https://www.facebook.com/'

def initialize_driver():
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_experimental_option(
        "prefs", {"profile.default_content_setting_values.notifications": 1}
    )
    return webdriver.Chrome(options=option)

def crawl_group_member(driver, group_list, start_index, chunk_size, result):
    group_crawler = group.Group(browser=driver)
    for i in range(start_index, start_index+chunk_size):
        try:
            user_list = group_crawler.get_url_from_members(group_list[i], max_user=100)
        except:
            user_list = None
        result[i] = user_list

def multi_thread_crawl(group_links,num_thread=5):

    drivers = [initialize_driver() for _ in range(5)]

    for driver in drivers:
        driver.get(base_url)
        login(driver, username, password)

    result = [None] * len(group_links)
    chunk_size = len(group_links) //  num_thread  # Max = 5
    start = 0
    threads = []
    for driver in drivers:
        thread = threading.Thread(target=crawl_group_member, args=(driver, group_links, start, chunk_size, result))
        threads.append(thread)
        start += chunk_size

    start_time = time.perf_counter()
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print(time.perf_counter() - start_time, 'seconds')

    for driver in drivers:
        driver.quit()

    return result